In [2]:
import tensorflow as tf
import random
import main
import numpy as np
from code import temp

In [207]:
def dynamicRNN( x,y,seqlen,vocab_size,seq_max_len):
    n_hidden = 50
    x_unstack = tf.unstack(x, seq_max_len, 1)
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, vocab_size],seed=1),name = 'w1')
    }

    biases = {
        'out': tf.Variable(tf.zeros([vocab_size]),name='b1')
    }

    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,name="lstm_cell")

    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x_unstack, dtype=tf.float32,
                                sequence_length=seqlen)
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])
    batch_size = tf.shape(outputs)[0] 
    outputs = tf.reshape(outputs, [-1, n_hidden])
    outputs  = tf.matmul(outputs, weights['out']) + biases['out']

    allOutputs = tf.nn.softmax(outputs)
    allOutputs = tf.reshape(allOutputs,[batch_size,seq_max_len,vocab_size],name="allOutputs")

    outputs = tf.reshape(outputs,[batch_size,seq_max_len,vocab_size])


    index = tf.range(0,batch_size)*seq_max_len + (seqlen -1)
    outputPred = tf.gather(tf.reshape(outputs,[-1,vocab_size]),index)
    outputPred = tf.nn.softmax(outputPred,name = "predictedOutputs")
    outputPred  = tf.argmax(outputPred,axis = -1)
    predictedOutputs = []
    outputPred =  tf.one_hot(outputPred,vocab_size,name ="one_hot")
    # states = tf.identity(states,name="states")

    for i in range(20):
        predictedOutputs.append(outputPred)
        outputPred, states = lstm_cell(outputPred,states)
        outputPred = tf.matmul(outputPred, weights['out']) + biases['out']
        # softmax_cross_entropy_with_logits
        outputPred = tf.nn.softmax(outputPred,name ="softymax")
        outputPred  = tf.argmax(outputPred,axis = -1,name="argymax")
        outputPred =  tf.one_hot(outputPred,vocab_size,name="one_hot_vec")



    predictedOutputs = tf.stack(predictedOutputs,name="predictedOutputing")	

    predictedOutputs =[]
    return outputs,predictedOutputs

In [191]:
def mainRun():
    tf.reset_default_graph()
    tf.set_random_seed(1)
    learning_rate = 0.01
    x1,y1,t,vocab_size  = main.main()
    seq_max_len = max(t)

    x = tf.placeholder("float", [None,seq_max_len, vocab_size],name="x")
    y = tf.placeholder("float", [None,seq_max_len, vocab_size],name= "y")
    seqlen = tf.placeholder(tf.int32, [None],name ="seqlen")

    pred,b = dynamicRNN(x,y,seqlen,vocab_size,max(t))

    # return pred,b
    # return pred,b
    init = tf.global_variables_initializer()
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y),name="theloss")
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,name="the_gdo").minimize(cost,name="theoptimizer")
    saver = tf.train.Saver()
    sess = tf.Session()
    # saver = tf.train.import_meta_graph('saved/model-10000.meta')
    # saver.restore(sess,tf.train.latest_checkpoint('saved/.'))

    sess.run(init)
    p  = sess.run(pred,feed_dict={x:x1,y:y1,seqlen:t})
    return p
    # graph = tf.get_default_graph()
    # pred = graph.get_tensor_by_name("b1:0")
    # print(sess.run(pred))
    # return

    # l,m  = sess.run([pred,b],feed_dict={x:x1,y:y1,seqlen:t})
    # return l,m,i

    for i in range(10001):
         l,o  = sess.run([cost,optimizer],feed_dict={x:x1,y:y1,seqlen:t})
         if not(i % 500):
            print("saving model")
            saver.save(sess,'./saved/SMallmodel',global_step = i)
         print("l {} o {} epoch {}".format(l,o,i)) 

In [208]:
a = mainRun()

./code/train/train1.java


In [206]:
print(a)

[[[-0.05028982 -0.15775123  0.02189661 ...  0.11746041  0.1399475
   -0.00259064]
  [ 0.16118541  0.31046888 -0.11133115 ... -0.08907773  0.0908433
    0.27278033]
  [-0.01634175  0.1995528  -0.13040361 ... -0.01535744 -0.13777074
    0.03633858]
  ...
  [-0.5151518  -0.35652903 -0.4848704  ... -0.03180604 -0.31240112
    0.5509635 ]
  [-0.5516953  -0.3896848  -0.55056477 ... -0.01627211 -0.31756756
    0.5401726 ]
  [-0.42702863 -0.5062691  -0.22947006 ... -0.03351505 -0.30154946
    0.18072931]]]


In [213]:
print(a)

[[[-0.05028982 -0.15775123  0.02189661 ...  0.11746041  0.1399475
   -0.00259064]
  [ 0.16118541  0.31046888 -0.11133115 ... -0.08907773  0.0908433
    0.27278033]
  [-0.01634175  0.1995528  -0.13040361 ... -0.01535744 -0.13777074
    0.03633858]
  ...
  [-0.5151518  -0.35652903 -0.4848704  ... -0.03180604 -0.31240112
    0.5509635 ]
  [-0.5516953  -0.3896848  -0.55056477 ... -0.01627211 -0.31756756
    0.5401726 ]
  [-0.42702863 -0.5062691  -0.22947006 ... -0.03351505 -0.30154946
    0.18072931]]]


In [293]:
reload(main)

<module 'main' from 'main.pyc'>

In [5]:
def mainContinue():
	tf.reset_default_graph()
	learning_rate = 0.01
	x1,y1,t,vocab_size  = main.main()
	sess = tf.Session()
	saver = tf.train.import_meta_graph('saved/SMallmodel-10000.meta')
	saver.restore(sess,tf.train.latest_checkpoint('saved/.'))
	graph = tf.get_default_graph()
	xP = graph.get_tensor_by_name("x:0")
	yP = graph.get_tensor_by_name("y:0")
	seqlenP = graph.get_tensor_by_name("seqlen:0")

	
	cost =graph.get_tensor_by_name("theloss:0")
	b = graph.get_tensor_by_name("b1:0")

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,name="the_gdo").minimize(cost,name="theoptimizer")
	
	
	for i in range(10000,50000):
		 l,_  = sess.run([cost,optimizer],feed_dict={xP:x1,yP:y1,seqlenP:t})
		 # print(l,b)
		 if not(i % 500):
			print("saving model")
		 	saver.save(sess,'./saved/SMallmodel',global_step = i)
		 print("l {}  epoch {}".format(l,i))

In [307]:
def restore():
    tf.reset_default_graph()
    token_code = temp.getCode()
    x1,y1,t,vocab_size = main.testMain(token_code)
#     x1,y1,t,vocab_size = main.main()

    word_to_id,id_to_word,token_docs = main.word_id()
    sess = tf.Session()
#     x1 = np.zeros((1, max(t), vocab_size))
#     idwod = word_to_id['import']
#     x1[:,:,idwod] =  1
#     t = [1]
    saver = tf.train.import_meta_graph('saved/SMallmodel-10000.meta')
    saver.restore(sess,tf.train.latest_checkpoint('saved/.'))
    graph = tf.get_default_graph()
    pred = graph.get_tensor_by_name("predictedOutputing:0")
    all_outputs  = graph.get_tensor_by_name("allOutputs:0")
    all_outputs  = tf.argmax(all_outputs,axis = -1,name="AllOargymax")
    all_outputs =  tf.one_hot(all_outputs,vocab_size,name="AllOone_hot_vec")
    predictedOutputs = []
    # for i in range(10):
    # 	outputPred, states = lstm_cell(outputsPred,states)
    # 	outputPred = tf.matmul(outputPred, weights['out']) + biases['out']
    # 	# softmax_cross_entropy_with_logits
    # 	outputPred = tf.nn.softmax(outputPred)
    # 	predictedOutputs.append(outputPred)
    # 	outputPred  = tf.argmax(outputPred) 
    # 	outputPred =  tf.one_hot(outputPred,vocab_size)


    # predictedOutputs = tf.stack(predictedOutputs,name="predictedOutputs")	
    xP = graph.get_tensor_by_name("x:0")
    yP = graph.get_tensor_by_name("y:0")
    seqlenP = graph.get_tensor_by_name("seqlen:0")
    cost =graph.get_tensor_by_name("theloss:0")
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=yP))	

    p,cost,all_outputs  = sess.run([pred,cost,all_outputs],feed_dict={xP:x1,seqlenP:t,yP:y1})
    pred = np.argmax(p,axis = 2)
    pred = pred.flatten()
    predArray = [id_to_word[i] for i in pred]
    predWord = ' '.join(predArray)
    aText = np.argmax(all_outputs,axis = 2)
    aText = aText.flatten()
    aText = [id_to_word[i] for i in aText]
    aText = ' '.join(aText)
    return p,aText,predWord,cost


In [298]:

tf.reset_default_graph()
token_code = temp.getCode()
print(token_code)
x1,y1,t,vocab_size = main.testMain(token_code)

('val', '12', 32)
1
('heck', 'import')
1
('heck', 'java')
1
('heck', '.')
1
('heck', 'util')
1
('heck', '.')
1
('heck', '*')
1
('heck', ';')
1
('heck', '\n')
2
('heck', 'import')
2
('heck', 'java')
2
('heck', '.')
2
('heck', 'lang')
2
('heck', '.')
2
('heck', '*')
2
('heck', ';')
2
('heck', '\n')
3
('heck', 'public')
3
('heck', 'class')
3
('heck', 'train1')
3
('heck', '\n')
4
('heck', '{')
4
('heck', '\n')
5
('heck', 'static')
5
('heck', 'ArrayList')
5
('heck', '<')
5
('heck', 'Integer')
5
('heck', '>')
5
('heck', 'array')
5
('heck', ';')
5
('heck', '\n')
6
('heck', 'public')
6
('heck', 'static')
6
('heck', 'void')
6
('heck', 'main')
6
('heck', '(')
6
('heck', 'String')
6
('heck', 'args')
6
('heck', '[')
6
('heck', ']')
6
('heck', ')')
6
('heck', '\n')
7
('heck', '{')
7
('heck', '\n')
8
('heck', 'array')
8
('heck', '=')
8
('heck', 'new')
8
('heck', 'ArrayList')
8
('heck', '<')
8
('heck', 'Integer')
8
('heck', '>')
8
('heck', '(')
8
('heck', ')')
8
('heck', ';')
8
('heck', '\n')
9
('hec

In [300]:
xCustom,yCustom,tCustom,vocab_sizeCustom = main.main()
# xCustom = np.zeros((1, max(tCustom), vocab_sizeCustom))
# word_to_id,id_to_word,token_docs = main.word_id()
# idwod = word_to_id['import']
# xCustom[:,:,idwod] =  1
# tCustom = [1]

./code/train/train1.java


In [91]:

sess = tf.Session()
saver = tf.train.import_meta_graph('saved/SMallmodel-10000.meta')
saver.restore(sess,tf.train.latest_checkpoint('saved/.'))
graph = tf.get_default_graph() 

INFO:tensorflow:Restoring parameters from saved/./SMallmodel-10000


In [132]:
pred = graph.get_tensor_by_name("predictedOutputing:0")
all_outputs  = graph.get_tensor_by_name("allOutputs:0")
all_outputs  = tf.argmax(all_outputs,axis = -1,name="AllOargymax")
all_outputs =  tf.one_hot(all_outputs,vocab_size,name="AllOone_hot_vec")
predictedOutputs = []
xP = graph.get_tensor_by_name("x:0")
yP = graph.get_tensor_by_name("y:0")
seqlenP = graph.get_tensor_by_name("seqlen:0")
cost =graph.get_tensor_by_name("theloss:0")

In [301]:

p,costVal,all_outputsVal  = sess.run([pred,cost,all_outputs],feed_dict={xP:x1,seqlenP:t,yP:y1})

In [302]:

pC,costValC,all_outputsValC  = sess.run([pred,cost,all_outputs],feed_dict={xP:xCustom,seqlenP:tCustom,yP:yCustom})

In [303]:

predArray = np.argmax(p,axis = 2)
predArray = predArray.flatten()
predArray = [id_to_word[i] for i in predArray]
aText = np.argmax(all_outputsVal,axis = 2)
aText = aText.flatten()
aText = [id_to_word[i] for i in aText]


predC= np.argmax(pC,axis = 2)
predC = predC.flatten()
predArrayC = [id_to_word[i] for i in predC]
aTextC = np.argmax(all_outputsValC,axis = 2)
aTextC = aTextC.flatten()
aTextC= [id_to_word[i] for i in aTextC]

In [304]:
aTextC = np.argmax(all_outputsValC,axis = 2)
aTextC = aTextC.flatten()
aTextC= [id_to_word[i] for i in aTextC]
print(aTextC)
print(costValC)
print(tCustom)

['java', '.', 'util', '.', '*', ';', '\n', 'import', 'java', '.', 'lang', '.', '*', ';', '\n', 'public', 'class', 'train1', '\n', '{', '\n', 'static', 'ArrayList', '<', 'Integer', '>', 'array', ';', '\n', 'public', 'static', 'void', 'main', '(', 'String', 'args', '[', ']', ')', '\n', '{', '\n', 'array', '=', 'new', 'ArrayList', '<', 'Integer', '>', '(', ')', ';', '\n', 'int', 'fact', '=', 'factorial', '(', '5', ')', ';', '\n', 'int', 'multiply', '=', 'multiply', '(', '5', ',', '10', ')', ';', '\n', 'binaryConver', '(', '8', ')', ';', '\n', 'for', '(', 'int', 'i', '=', '0', ';', 'i', '<', 'array', '.', 'size', '(', ')', ';', 'i', '+', '+', ')', '{', '\n', 'System', '.', 'out', '.', 'println', '(', 'array', '.', 'get', '(', 'i', ')', ')', ';', '\n', '}', '\n', 'System', '.', 'out', '.', 'println', '(', '"factorial is "', '+', 'fact', ')', ';', '\n', 'int', 'val', '=', 'power', '(', '3', ',', '4', ')', ';', '\n', 'System', '.', 'out', '.', 'println', '(', '"power val is "', '+', 'val', ')

In [305]:
print(aText)
print(costVal)
print(t)

['java', '.', 'util', '.', '*', ';', '\n', 'import', 'java', '.', 'lang', '.', '*', ';', '\n', 'public', 'class', 'train1', '\n', '{', '\n', 'static', 'ArrayList', '<', 'Integer', '>', 'array', ';', '\n', 'public', 'static', 'void', 'main', '(', 'String', 'args', '[', ']', ')', '\n', '{', '\n', 'array', '=', 'new', 'ArrayList', '<', 'Integer', '>', '(', ')', ';', '\n', 'int', 'fact', '=', 'factorial', '(', '5', ')', ';', '\n', 'int', 'multiply', '=', 'multiply', '(', '5', ',', '10', ')', ';', '\n', 'binaryConver', '(', '8', ')', ';', '\n', 'for', '(', 'int', 'i', '=', '0', ';', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java', 'java',

In [227]:
print(x1)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [291]:
import numpy as np
x2 =  [np.argmax(np.array(i)) for i in x1[0]]
print(x2[:78])

x2C =  [np.argmax(np.array(i)) for i in xCustom[0]]
print(x2C[:78])

[40, 10, 19, 38, 19, 16, 26, 40, 10, 19, 45, 19, 16, 26, 51, 44, 24, 55, 5, 50, 29, 25, 30, 8, 26, 51, 5, 35, 27, 14, 11, 36, 42, 43, 13, 55, 8, 28, 37, 50, 29, 25, 30, 14, 13, 26, 1, 57, 28, 34, 14, 23, 13, 26, 1, 41, 28, 41, 14, 23, 18, 46, 13, 26, 7, 14, 39, 13, 26, 12, 14, 1, 48, 28, 21, 0, 0, 0]
[40, 10, 19, 38, 19, 16, 26, 2, 40, 10, 19, 45, 19, 16, 26, 2, 51, 44, 24, 2, 55, 2, 5, 50, 29, 25, 30, 8, 26, 2, 51, 5, 35, 27, 14, 11, 36, 42, 43, 13, 2, 55, 2, 8, 28, 37, 50, 29, 25, 30, 14, 13, 26, 2, 1, 57, 28, 34, 14, 23, 13, 26, 2, 1, 41, 28, 41, 14, 23, 18, 46, 13, 26, 2, 7, 14, 39, 13]


In [53]:
print(len(x1[0][0]))

58


In [308]:
restore()

('val', '12', 32)
1
('heck', 'import')
1
('heck', 'java')
1
('heck', '.')
1
('heck', 'util')
1
('heck', '.')
1
('heck', '*')
1
('heck', ';')
1
('heck', '\n')
2
('heck', 'import')
2
('heck', 'java')
2
('heck', '.')
2
('heck', 'lang')
2
('heck', '.')
2
('heck', '*')
2
('heck', ';')
2
('heck', '\n')
3
('heck', 'public')
3
('heck', 'class')
3
('heck', 'train1')
3
('heck', '\n')
4
('heck', '{')
4
('heck', '\n')
5
('heck', 'static')
5
('heck', 'ArrayList')
5
('heck', '<')
5
('heck', 'Integer')
5
('heck', '>')
5
('heck', 'array')
5
('heck', ';')
5
('heck', '\n')
6
('heck', 'public')
6
('heck', 'static')
6
('heck', 'void')
6
('heck', 'main')
6
('heck', '(')
6
('heck', 'String')
6
('heck', 'args')
6
('heck', '[')
6
('heck', ']')
6
('heck', ')')
6
('heck', '\n')
7
('heck', '{')
7
('heck', '\n')
8
('heck', 'array')
8
('heck', '=')
8
('heck', 'new')
8
('heck', 'ArrayList')
8
('heck', '<')
8
('heck', 'Integer')
8
('heck', '>')
8
('heck', '(')
8
('heck', ')')
8
('heck', ';')
8
('heck', '\n')
9
('hec

(array([[[0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.]]], dtype=float32),
 'java . util . * ; \n import java . lang . * ; \n public class train1 \n { \n static ArrayList < Integer > array ; \n public static void main ( String args [ ] ) \n { \n array = new ArrayList < Integer > ( ) ; \n int fact = factorial ( 5 ) ; \n int multiply = multiply ( 5 , 10 ) ; \n binaryConver ( 8 ) ; \n for ( int i = 0 ; java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java java

In [146]:
input = ['import', 'java', '.', 'util', '.', '*', ';', '\n', 'import', 'java', '.', 'lang', '.', '*', ';', '\n', 'public', 'class', 'train1', '\n', '{', '\n', 'static', 'ArrayList', '<', 'Integer', '>', 'array', ';', '\n', 'public', 'static', 'void', 'main', '(', 'String', 'args', '[', ']', ')', '\n', '{', '\n', 'array', '=', 'new', 'ArrayList', '<', 'Integer', '>', '(', ')', ';', '\n', 'int', 'fact', '=', 'factorial', '(', '5', ')', ';', '\n', 'int', 'multiply', '=', 'multiply', '(', '5', ',', '10', ')', ';', '\n', 'binaryConver', '(', '8', ')', ';', '\n', 'for', '(', 'int', 'i', '=', '0', ';', 'i', '<']